# Constant

In [1]:
import certifi
import json
import time
import random
import psycopg2
import os
from psycopg2.extras import execute_values

API_KEY = "f8fe2458856a3904f2a649ebf1e82a68"
PG_HOST = os.environ.get('POSTGRES_HOST','')
PG_PORT = os.environ.get('POSTGRES_PORT','')
PG_DB = os.environ.get('POSTGRES_DB','')
PG_USER = os.environ.get('POSTGRES_USER','')
PG_PASS = os.environ.get('POSTGRES_PASSWORD','')

# Common library

In [2]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    time.sleep(1)
    return json.loads(data)

def emptyToNull(val):
    if val:
        return val
    else:
        return None

def connectDB():
    con = psycopg2.connect(user=PG_USER,
                       password=PG_PASS,
                       host=PG_HOST,
                       port=PG_PORT,
                       database=PG_DB)
    return con

# Global variable

In [3]:
dbCon=connectDB()
dbCon.autocommit = True
dbCur=dbCon.cursor()

# Get symbol list

In [4]:
url = ("https://financialmodelingprep.com/api/v3/stock/list?apikey="+API_KEY)
retDict = get_jsonparsed_data(url)
symbol_dat = []

if retDict:
    for item in retDict[0:]:
        symbol_dat.append((item.get('symbol'), item.get('name')))
    execute_values(dbCur, "INSERT INTO public.stock (symbol, stockname) VALUES %s ON CONFLICT (symbol) DO UPDATE SET stockname=EXCLUDED.stockname, updated_time=NOW()", symbol_dat)
#print(symbol_dat)

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


# Get delisted companies

In [6]:
startpage = 0
maxpage = 3
delistedCom_dat = []

for page in range(startpage,  maxpage):
    url = ("https://financialmodelingprep.com/api/v3/delisted-companies?page="+str(page)+"&apikey="+API_KEY)
    retDict=get_jsonparsed_data(url)
    if retDict:
        print("page:", page)
        for item in retDict:
            symbol = item.get('symbol')
            symbol_list = [x[0] for x in symbol_dat]
            if symbol in symbol_list:
                delistedCom_dat.append((item.get('symbol'), item.get('companyName'), item.get('exchange'), item.get('ipoDate'), item.get('delistedDate')))
        execute_values(dbCur, "INSERT INTO public.delistedcompany (symbolid, companyname, exchange, ipodate, delisteddate) VALUES %s ON CONFLICT ON CONSTRAINT delistedcompany_uq\
        DO UPDATE SET  companyname=EXCLUDED.companyname, exchange=EXCLUDED.exchange, ipodate=EXCLUDED.ipodate, delisteddate=EXCLUDED.delisteddate, updated_time=NOW()", delistedCom_dat)
    else:
        break

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


page: 0
page: 1
page: 2


# Get historical dividends

In [7]:
dividend_dat = []
for symbol, symbolName in symbol_dat[0:]:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/"+symbol+"?apikey="+API_KEY)
    retDict = get_jsonparsed_data(url)
    if retDict:
        for item in retDict.get('historical'):
            dividend_dat.append((symbol, item.get('date'), item.get('label'), item.get('adjDividend'), item.get('dividend'), emptyToNull(item.get('recordDate')), emptyToNull(item.get('paymentDate')), emptyToNull(item.get('declarationDate'))))
        #print(dividend_dat)
        execute_values(dbCur, "INSERT INTO public.historicaldividend (symbolid, dividenddate, label, adjdividend, dividend, recorddate, paymentdate, declarationdate) VALUES %s ON CONFLICT  ON CONSTRAINT historicaldividend_uq  \
        DO UPDATE SET dividenddate=EXCLUDED.dividenddate, label=EXCLUDED.label, adjdividend=EXCLUDED.adjdividend, dividend=EXCLUDED.dividend, recorddate=EXCLUDED.recorddate, paymentdate=EXCLUDED.paymentdate, declarationdate=EXCLUDED.declarationdate, updated_time=NOW()", dividend_dat)
    else:
        break

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


# Clean whatever we need to

In [9]:
dbCur.close()
dbCon.close()